In [4]:
import pandas as pd

file = "ais_interpolated.csv"
df = pd.read_csv(file)
print(f"Initial number of rows: {len(df)}")
df.dropna(subset=["Latitude", "Longitude"], inplace=True)
print(f"Number of rows after dropping NA: {len(df)}")
df

Initial number of rows: 124141
Number of rows after dropping NA: 114789


,MMSI,Timestamp,Latitude,Longitude,SOG,COG
0,203247191,2025-10-27 00:10:00,54.793802,9.434013,0.051444,NaN
1,203247191,2025-10-27 00:20:00,54.793803,9.434028,0.000000,NaN
2,203247191,2025-10-27 00:30:00,54.793801,9.434001,0.024716,NaN
3,203247191,2025-10-27 00:40:00,54.793830,9.433935,0.051444,NaN
4,203247191,2025-10-27 00:50:00,54.793825,9.433992,0.000000,NaN
...,...,...,...,...,...,...
124136,750399000,2025-10-27 04:30:00,55.632529,15.639479,6.584883,74.900000
124137,750399000,2025-10-27 04:40:00,55.641399,15.699847,6.481994,75.027273
124138,750399000,2025-10-27 04:50:00,55.650020,15.759456,6.379106,75.120000
124139,750399000,2025-10-27 05:00:00,55.658270,15.818228,6.309929,75.379574


In [5]:
boat_ids = df['MMSI'].unique()
selected_boats = boat_ids[:5]  # Select first 5 boats for demonstration
subset_df = df[df['MMSI'].isin(selected_boats)]
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
          'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
          'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
center_lat = subset_df['Latitude'].mean()
center_lon = subset_df['Longitude'].mean()


In [8]:
import folium
import math
from folium_arrow_icon import ArrowIcon

# basic map
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=8)

for i, mmsi in enumerate(selected_boats):
    boat_df = df[df['MMSI'] == mmsi]
    color = colors[i % len(colors)]

    for _, row in boat_df.iterrows():
        # Prepare popup HTML content (for both branches)
        popup_content = f"""
            <b>MMSI:</b> {row['MMSI']}<br>
            <b>Date:</b> {row['Timestamp']}<br>
            <b>Speed:</b> {row['SOG']}<br>
            <b>Longitude:</b> {row['Longitude']}<br>
            <b>Latitude:</b> {row['Latitude']}<br>
            <b>COG:</b> {row['COG']}
        """
        if not pd.isna(row['COG']):
            # Arrow marker code
            angle = math.radians((90 - row['COG']) % 360)
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                icon=ArrowIcon(30, angle, color=color),
                popup=folium.Popup(popup_content, max_width=300),
                tooltip=f"MMSI: {row['MMSI']} | Date: {row['Timestamp']}"
            ).add_to(m)
        else:
            # Circle marker code
            folium.CircleMarker(
                location=[row['Latitude'], row['Longitude']],
                radius=6,
                color=color,
                fill=True,
                fill_color=color,
                popup=folium.Popup(popup_content, max_width=300),
                tooltip=f"MMSI: {row['MMSI']} | Date: {row['Timestamp']}"
            ).add_to(m)

folium.LayerControl().add_to(m)
m
